# verify_environment.ipynb
A small notebook to ensure that the correct libraries are installed for the user. 

## Libraries to test

In [1]:
import control as ct
import numpy as np
import plotly
from plotly.subplots import make_subplots

import ipywidgets as widgets
from IPython.display import display


## Helper Functions
Used for building the system and plotting

In [2]:
# mapping of the different response types
response_map = {
    "step": ct.step_response,
    "initial": ct.initial_response,
    "forced": ct.forced_response
}

In [3]:
# m - mass element, c - damping element, k - spring element
DEFAULT_M, DEFAULT_C, DEFAULT_K = 1.0, 0.1, 2.0

def build_system(m, c, k):
    # dx/dt = Ax + Bu, y = Cx + Du
    A = np.array([
        [0,      0,      1,     0],
        [0,      0,      0,     1],
        [-2*k/m,  k/m,  -c/m,   0],
        [ k/m,  -2*k/m,  0,    -c/m]
    ])
    B = np.array([[0], [0], [0], [k/m]])
    C = np.array([
        [1, 0, 0, 0],   # q1
        [0, 1, 0, 0]    # q2
    ])
    D = np.array([[0], [0]])

    return ct.ss(A, B, C, D)


In [4]:
def plot_response(response_type="step", m=DEFAULT_M, c=DEFAULT_C, k=DEFAULT_K):
    sys = build_system(m, c, k)
    T = np.linspace(0, 20, 400)

    if response_type == "initial":
        resp0 = ct.initial_response(sys, T=T, X0=[1, 0, 0, 0])
    elif response_type == "forced":
        U = np.ones_like(T)
        resp0 = ct.forced_response(sys, T=T, U=U)
    else:
        resp0 = ct.step_response(sys, T=T)

    t0 = resp0.time
    y0 = resp0.outputs
    if y0.ndim == 3:
        # step_response returns (outputs, inputs, time); select the single input
        y0 = y0[:, 0, :]

    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        subplot_titles=("q1 displacement", "q2 displacement")
    )

    fig.add_trace(
        plotly.graph_objects.Scatter(x=t0, y=y0[0], name="q1", mode="lines"),
        row=1, col=1
    )

    fig.add_trace(
        plotly.graph_objects.Scatter(x=t0, y=y0[1], name="q2", mode="lines"),
        row=2, col=1
    )

    fig.update_layout(
        title=f"{response_type.title()} Response (m={m:.2f}, c={c:.2f}, k={k:.2f})",
        template="plotly_white",
        height=600
    )

    fig.update_xaxes(title_text="Time (s)", row=2, col=1)
    fig.update_yaxes(title_text="q1", row=1, col=1)
    fig.update_yaxes(title_text="q2", row=2, col=1)

    display(fig)


## Small Example
This notebook adapts Example 1 from the Python Control Systems Library tutorial:
https://python-control.readthedocs.io/en/0.10.2/examples/python-control_tutorial.html

We use the same coupled mass–spring–damper model and modify the presentation to add interactive Plotly visualizations.


An interactive plot where someone can see the adjustment in response based on the changing system parameters

The reponse type is adjustable to "step", "initial", or "forced"

In [ ]:
# initializes the sliders
m_slider = widgets.FloatSlider(value=DEFAULT_M, min=0.5, max=5.0, step=0.1, description="m")
c_slider = widgets.FloatSlider(value=DEFAULT_C, min=0.0, max=2.0, step=0.05, description="c")
k_slider = widgets.FloatSlider(value=DEFAULT_K, min=0.5, max=10.0, step=0.1, description="k")

response_dropdown = widgets.Dropdown(
    options=["step", "initial", "forced"],
    value="step",
    description="response"
)

ui = widgets.VBox([response_dropdown, m_slider, c_slider, k_slider])
out = widgets.interactive_output(
    plot_response,
    {"response_type": response_dropdown, "m": m_slider, "c": c_slider, "k": k_slider}
)
display(ui, out)


Output()